- Text Data: 
One-hot encodding, Tokenize, word index, text to numeric

In [ ]:
from keras.preprocessing.text import Tokenizer
samples = ['The cat sat on the mat.', 'The dog ate my homework. ']

tokenizer = Tokenizer(num_words=100)
tokenizer.fit_on_texts(samples)

sequences = tokenizer.texts_to_sequences(samples)

one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary' )
word_index = tokenizer.word_index

print('sequences:' , sequences)
print('one_hot_results:' , one_hot_results[0])
print('word_index:' , word_index)
print('Found %s unique tokens.' % len(word_index) )



sequences: [[1, 2, 3, 4, 1, 5], [1, 6, 7, 8, 9]]
one_hot_results: [0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
word_index: {'the': 1, 'cat': 2, 'sat': 3, 'on': 4, 'mat': 5, 'dog': 6, 'ate': 7, 'my': 8, 'homework': 9}
Found 9 unique tokens.


- Word Embedding with IMDB movie review data

In [ ]:
from keras.datasets import imdb
from keras.utils import pad_sequences
max_features = 10000
maxlen = 20

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)


In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding

model = Sequential()
model.add(Embedding(10000, 8, input_length=maxlen) )
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

history = model.fit(x_train, y_train,
epochs=10,
batch_size=32,
validation_split=0.2)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 8)             80000     
                                                                 
 flatten (Flatten)           (None, 160)               0         
                                                                 
 dense (Dense)               (None, 1)                 161       
                                                                 
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 2s 2ms/step - loss: 0.6635 - acc: 0.6289 - val_loss: 0.6049 - val_acc: 0.7016
Epoch 2/10
625/625 [==============================] - 1s 2ms/step - loss: 0.5297 - acc: 0.7575 - val_loss: 0.5174 - val_acc: 0.7356
Epoch 3/10
625/625 [==============================] 

-----------------------------------------------
- For Better Performance we can use pre-trained word embeddings

- Download, Extract and load IMDB data from scratch

In [2]:
!wget http://mng.bz/0tIo #download the data file

!unzip 0tIo # extract the file

-----------------------------------------
- Load and save data in texts and labels variables

In [ ]:
import os
import pandas as pd

#imdb_dir = '/Users/fchollet/Downloads/aclImdb' # http://mng.bz/0tIo
train_dir = os.path.join('aclImdb/', 'train')
labels = []
texts = []

for label_type in ['neg', 'pos']:
  dir_name = os.path.join(train_dir, label_type)
  for fname in os.listdir(dir_name):
    if fname[-4:] == '.txt':
      f = open(os.path.join(dir_name, fname) )
      texts.append(f.read())
      f.close()
      if label_type == 'neg':
        labels.append (0)
      else:
        labels.append(1)

-----------------------------
- Tokenize and Vectorize data

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
import numpy as np

maxlen = 100
training_samples = 200
validation_samples = 10000
max_words = 10000

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index) )

data = pad_sequences(sequences, maxlen=maxlen)
labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]


x_train = data[:training_samples]
y_train = labels[:training_samples]
X_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

Found 88582 unique tokens.
Shape of data tensor: (25000, 100)
Shape of label tensor: (25000,)


------------------------------
- Using GLoVe for Embedding Layer
Download and Extract GLoVe

In [ ]:
!wget https://nlp.stanford.edu/data/glove.6B.zip #download the data file

!unzip glove.6B.zip # extract the file

Archive:  glove.6B.zip.1
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
import os
import numpy as np
#glove_dir = 'glove.6B'
embeddings_index = {}
f = open(os.path.join('', 'glove.6B.100d.txt'))
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  embeddings_index[word] = coefs
f.close()

-----------------------------------
- Specialize embedding vectors in GLoVe to our IMDB data

In [ ]:
max_words = 10000
embedding_dim = 100
embedding_matrix = np.zeros((max_words, embedding_dim))

for word, i in word_index.items():
  if(i < max_words):
    embedding_vector = embeddings_index.get(word)
    if(embedding_vector is not None):
      embedding_matrix[i] = embedding_vector

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary ()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          1000000   
                                                                 
 flatten (Flatten)           (None, 10000)             0         
                                                                 
 dense (Dense)               (None, 32)                320032    
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1,320,065
Trainable params: 1,320,065
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.layers[0].set_weights( [embedding_matrix] )
model.layers[0].trainable = False

In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

history = model.fit(x_train, y_train,
epochs=10,
batch_size=32,
validation_data=(X_val, y_val))

model.save_weights('pre_trained_glove_model.h5')

Epoch 1/10
7/7 [==============================] - 3s 247ms/step - loss: 1.2500 - acc: 0.4850 - val_loss: 0.6929 - val_acc: 0.5114
Epoch 2/10
7/7 [==============================] - 1s 111ms/step - loss: 0.6501 - acc: 0.6800 - val_loss: 0.6915 - val_acc: 0.5297
Epoch 3/10
7/7 [==============================] - 1s 113ms/step - loss: 0.6138 - acc: 0.6200 - val_loss: 0.8936 - val_acc: 0.5028
Epoch 4/10
7/7 [==============================] - 1s 112ms/step - loss: 0.6006 - acc: 0.7050 - val_loss: 0.6958 - val_acc: 0.5473
Epoch 5/10
7/7 [==============================] - 1s 221ms/step - loss: 0.4171 - acc: 0.7650 - val_loss: 0.7091 - val_acc: 0.5522
Epoch 6/10
7/7 [==============================] - 1s 109ms/step - loss: 0.2622 - acc: 0.9000 - val_loss: 1.3104 - val_acc: 0.5028
Epoch 7/10
7/7 [==============================] - 1s 113ms/step - loss: 0.1794 - acc: 0.9550 - val_loss: 0.8739 - val_acc: 0.5155
Epoch 8/10
7/7 [==============================] - 1s 108ms/step - loss: 0.2045 - acc: 0.94

- Evaluation	on	the	test	data

In [ ]:
test_dir = os.path.join('aclImdb/', 'test')
labels = []
texts = []
for label_type in ['neg', 'pos']:
  dir_name = os.path.join(test_dir, label_type)
  for fname in sorted(os.listdir(dir_name)):
    if(fname[-4:] == '.txt'):
      f = open(os.path.join(dir_name, fname) )
      texts .append(f.read())
      f.close()
      if label_type == 'neg':
        labels.append(0)
      else:
        labels.append(1)

sequences = tokenizer.texts_to_sequences(texts)
x_test = pad_sequences(sequences, maxlen=maxlen)
y_test = np.asarray(labels)
model.load_weights('pre_trained_glove_model.h5')
model.evaluate(x_test, y_test)

782/782 [==============================] - 2s 2ms/step - loss: 0.7367 - acc: 0.5697


[0.7366601824760437, 0.5697199702262878]

------------------------------------
- Numpy implementation	of	a	simple	RNN


In [ ]:
import numpy as np
timesteps = 100
input_features = 32

output_features = 64

inputs = np.random.random((timesteps, input_features))
state_t = np.zeros((output_features,))

W = np.random.random((output_features, input_features))
U = np.random.random((output_features, output_features))
b = np.random.random((output_features, ))

successive_outputs = []

for input_t in inputs:
  output_t = np.tanh(np.dot(W, input_t) + np.dot(U, state_t) + b)
  successive_outputs.append(output_t)
  state_t = output_t

final_output_sequence = np.array(successive_outputs)

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding, SimpleRNN

max_features = 10000
maxlen = 500
model = Sequential()

model.add(Embedding(max_features, 100))
model.add(SimpleRNN(100))
model.add(Dense(1, activation='sigmoid') )

model.layers[0].set_weights( [embedding_matrix] )
model.layers[0].trainable = False
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

history = model.fit(x_train, y_train,
epochs=10,
batch_size=128,
validation_split=0.2)

Epoch 1/15
2/2 [==============================] - 3s 323ms/step - loss: 0.7849 - acc: 0.5000 - val_loss: 0.6663 - val_acc: 0.5500
Epoch 2/15
2/2 [==============================] - 0s 73ms/step - loss: 0.6161 - acc: 0.6625 - val_loss: 0.6273 - val_acc: 0.5750
Epoch 3/15
2/2 [==============================] - 0s 65ms/step - loss: 0.5633 - acc: 0.7563 - val_loss: 0.6413 - val_acc: 0.6500
Epoch 4/15
2/2 [==============================] - 0s 73ms/step - loss: 0.5138 - acc: 0.8062 - val_loss: 0.6460 - val_acc: 0.6500
Epoch 5/15
2/2 [==============================] - 0s 66ms/step - loss: 0.4756 - acc: 0.8438 - val_loss: 0.6310 - val_acc: 0.5750
Epoch 6/15
2/2 [==============================] - 0s 67ms/step - loss: 0.4431 - acc: 0.8438 - val_loss: 0.7149 - val_acc: 0.5750
Epoch 7/15
2/2 [==============================] - 0s 63ms/step - loss: 0.4233 - acc: 0.8500 - val_loss: 0.6561 - val_acc: 0.6750
Epoch 8/15
2/2 [==============================] - 0s 63ms/step - loss: 0.3685 - acc: 0.9187 - va

-----------------------
- LSTM

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding, LSTM

max_features = 10000
maxlen = 500
model = Sequential()

model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid') )

#model.layers[0].set_weights( [embedding_matrix] )
#model.layers[0].trainable = False
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

history = model.fit(x_train, y_train,
epochs=15,
batch_size=128,
validation_split=0.2)

Epoch 1/15
2/2 [==============================] - 3s 590ms/step - loss: 0.6938 - acc: 0.4750 - val_loss: 0.6917 - val_acc: 0.6000
Epoch 2/15
2/2 [==============================] - 0s 90ms/step - loss: 0.6883 - acc: 0.7312 - val_loss: 0.6899 - val_acc: 0.5250
Epoch 3/15
2/2 [==============================] - 0s 97ms/step - loss: 0.6827 - acc: 0.7688 - val_loss: 0.6876 - val_acc: 0.5250
Epoch 4/15
2/2 [==============================] - 0s 75ms/step - loss: 0.6749 - acc: 0.7812 - val_loss: 0.6847 - val_acc: 0.6250
Epoch 5/15
2/2 [==============================] - 0s 70ms/step - loss: 0.6624 - acc: 0.8813 - val_loss: 0.6778 - val_acc: 0.6000
Epoch 6/15
2/2 [==============================] - 0s 75ms/step - loss: 0.6379 - acc: 0.8813 - val_loss: 0.6556 - val_acc: 0.5500
Epoch 7/15
2/2 [==============================] - 0s 75ms/step - loss: 0.5479 - acc: 0.5938 - val_loss: 0.6681 - val_acc: 0.6250
Epoch 8/15
2/2 [==============================] - 0s 74ms/step - loss: 0.5696 - acc: 0.9750 - va